In [1]:
# 选择根目录
%cd /home/zjdou/zpwang/InsultingLanguageDetection/

model_name = 'bert-base-uncased'

sentence = 'hello world'

/home/zjdou/zpwang/InsultingLanguageDetection


In [2]:
from transformers import AutoTokenizer

# 创建分词器
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path=model_name,  # 模型名称
    cache_dir='./pretrained_model',  # 模型保存地址，可忽略
)

/home/zjdou/anaconda3/envs/wzp_main/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sentence_tokens = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')
sentences_tokens = tokenizer(
    text=[sentence, sentence, sentence],
    padding=True, truncation=True, return_tensors='pt'
)  # 同时分词多句话
print(sentence_tokens)  # 分词后的句子
input_size = sentence_tokens['input_ids'].size()  # 输入的尺寸
batch_size = input_size[0]  # 一次前馈的输入称为一个 batch，batch中样例个数称为 batch size
sequence_length = input_size[1]  # 分词后长度
print(input_size)
print(batch_size)
print(sequence_length)

{'input_ids': tensor([[ 101, 7592, 2088,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
torch.Size([1, 4])
1
4


In [4]:
from transformers import AutoModel

model = AutoModel.from_pretrained(
    pretrained_model_name_or_path=model_name,  # 模型名称
    cache_dir='./pretrained_model/',  # 模型保存地址，可忽略
)
model  # 查看模型结构

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [5]:
y = model(**sentence_tokens)  # 将sentence_tokens解码输入模型，含义同下面代码
# y = model(
#     input_ids=sentence_tokens['input_ids'],
#     token_type_ids=sentence_tokens['token_type_ids'],
#     attention_mask=sentence_tokens['attention_mask'],
# )
# y = model(**sentences_tokens)  # 多组输入的模型前馈方法
# print(y)
token_embedding = y.last_hidden_state  # 一般取last_hidden_state作为编码后的token表示
print(token_embedding.size())  # 输出维度：batch_size, sequence_length, model_output_dim模型输出维度（bert为768）
sentence_embedding = token_embedding[:, 0]  # 一般取last_hidden_state的第一个token编码作为句子表示
print(sentence_embedding.size())  # 输出维度：batch_size, model_output_dim模型输出维度（bert为768）



torch.Size([1, 4, 768])
torch.Size([1, 768])


## 更新